# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимпо построить модель со значением метрики качества *F1* не меньше 0.75. 

**Стадии выполнения проекта**

1. Загрузим и подготовим данные.
2. Обучим модель. 
3. Сделаем выводы.

Для выполнения проекта применим *BERT*.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходжимые библиотеки

In [1]:
import torch
import transformers
import pandas as pd
import numpy as np
from tqdm import notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

Загрузим наш датасет. Отберем 2000 текстов

In [2]:
df = pd.read_csv('toxic_comments.csv', usecols=[1, 2])

In [3]:
df_part_1 = df.sample(2000, random_state = 23)

### Изучение общей информации о датасете

Взглянем на общую информацию в нашем датасете

In [4]:
df_part_1.head()

,text,toxic
30309,"""\n\nIt's already there in the """"Fan Reaction""...",0
101384,"""\n\nI seem to recall that this is true and at...",0
109781,A guy from NET is harassing me\n\nThis guy htt...,0
42795,"Not really an error, if you notice it says the...",0
49160,"Sorry for not replying, MH - I did not see any...",0


In [5]:
df_part_1.shape

(2000, 2)

Посмотрим на баланс классов

In [6]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Присутствует дисбаланс классов. Для модели логистичесткой регрессии будем использовать параметр class_weight='balanced'

### Создадим эмбединги

In [7]:
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

In [8]:
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')

Проверим максимальную длину текстов в датасете. Ограничим длину текстов 512 символами для токенизации

In [9]:
max_len = df_part_1['text'].apply(
    lambda x: len(x)
    ).max()
print(f'максимальна длина текста: {max_len}')

максимальна длина текста: 4999


In [10]:
new_max_len = 512

In [11]:
df_part_1['text'] = df_part_1['text'].apply(
    lambda x: x[:new_max_len]
    )

Выполним токенизацию

In [12]:
tokenized_df_part_1 = df_part_1['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True)
    )

Выполним padding для коротких векторов

In [13]:
padded_df_part_1 = tokenized_df_part_1.apply(
    lambda x: np.array(x + [0]*(new_max_len-len(x)))
    )

Проверим выполнение padding-а

In [14]:
padded_df_part_1.apply(lambda x: len(x) < 512).sum()

0

Приведем данные к 2D array.

In [15]:
padded_df_part_1 = np.stack(padded_df_part_1)

Проверим размер датасета

In [16]:
padded_df_part_1.shape

(2000, 512)

Создадим маску attantion_mask

In [17]:
attention_mask = np.where(padded_df_part_1 != 0, 1, 0)
attention_mask.shape

(2000, 512)

Приступим к embedding-у

In [18]:
batch_size = 50

embeddings = []

for i in notebook.tqdm(range(padded_df_part_1.shape[0] // batch_size)):
  # преобразуем данные
  batch = torch.LongTensor(padded_df_part_1[batch_size*i:batch_size*(i+1)])
  # преобразуем маску
  attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
  
  with torch.no_grad():
    batch_embeddings = model(batch, attention_mask=attention_mask_batch)

  # преобразуем элементы методом numpy() к типу numpy.array
  embeddings.append(batch_embeddings[0][:,0,:].numpy())

features = np.concatenate(embeddings) 

  0%|          | 0/40 [00:00<?, ?it/s]

In [19]:
target = df_part_1['toxic']

### Разобъем features и target на выборки train и test

In [20]:
train_features, test_features, train_target, test_target = \
  train_test_split(features, target, test_size=.5, random_state=23, stratify=target)

### Итоги подготовки данных
- Данные были загружены
- Была создана выборка из 2000 текстов в корпусе
- Эмбединги были созданы при помощи предобученной модели BERT 'unitary/toxic-bert'
- Длина текстов была ограничена 512-ю символами
- Данные были разбиты на выборки train и test

## Обучение

Для создания предсказаний будем использовать модель логистическая регрессия

In [21]:
lr_model = LogisticRegression(max_iter=300, class_weight='balanced')
lr_model.fit(train_features, train_target)

LogisticRegression(class_weight='balanced', max_iter=300)

Создадим предсказания

In [22]:
predictions_lr = lr_model.predict(test_features)

Посчитаем метрику F1

In [23]:
print(f'F1 = {f1_score(test_target, predictions_lr):.3f}')

F1 = 0.873


### Итоги обучения модели

- В результате работы модели LogisticRegression было получено значение метрики F1 = 0.873

## Выводы

В результате подготовки данных:
- Данные были загружены
- Была создана выборка из 2000 текстов в корпусе
- Эмбединги были созданы при помощи предобученной модели BERT 'unitary/toxic-bert'
- Длина текстов была ограничена 512-ю символами
- Данные были разбиты на выборки train и test

В результате обучения модели:
- Было получено значение метрики F1 = 0.873 для модели LogisticRegression